In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from meteostat import Stations, Daily
from tqdm import tqdm
import time
warnings.filterwarnings("ignore")

In [ ]:
def load_csv_file(filename):
    return pd.read_csv(filename)
fd_train = load_csv_file("data/Train.csv")
fd_test =  load_csv_file("data/Test.csv")

In [ ]:
fd_train.describe()

In [ ]:
def get_commmon_columns():
    return ['id', 'date', 'flight_id', 'departure_point', 'arrival_point', 'departure_time', 'arrival_time', 'flight_status', 'aircraft_code']

def rename_colums(fd_train, extra_columns):
    columns = get_commmon_columns() + extra_columns
    fd_train.columns = columns
    return fd_train

fd_train = rename_colums(fd_train, ["target"])
fd_test = rename_colums(fd_test,[])

In [ ]:
def add_column_duration(fd_train):
    fd_train['departure_time'] = pd.to_datetime(fd_train['departure_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
    fd_train['arrival_time'] = pd.to_datetime(fd_train['arrival_time'], format='%Y-%m-%d %H.%M.%S', errors='coerce')
    fd_train['duration'] = (pd.to_datetime(fd_train['arrival_time']) - pd.to_datetime(fd_train['departure_time'])).dt.total_seconds()
    return fd_train

In [ ]:
def get_season(month):
    """Map month to season number."""
    if month in [12, 1, 2]:
        return 1  # Winter
    elif month in [3, 4, 5]:
        return 2  # Spring
    elif month in [6, 7, 8]:
        return 3  # Summer
    else:
        return 4  # Fall

def get_time_of_day(hour):
    """Categorize hour into time of day."""
    if 5 <= hour < 12:
        return 'morning'
    elif 12 <= hour < 17:
        return 'afternoon'
    elif 17 <= hour < 21:
        return 'evening'
    else:
        return 'night'

def add_datetime_features(df):
    df['departure_time'] = pd.to_datetime(df['departure_time'], errors='coerce')
    df['arrival_time'] = pd.to_datetime(df['arrival_time'], errors='coerce')

    # Duration in seconds
    df['duration'] = (df['arrival_time'] - df['departure_time']).dt.total_seconds()

    # Departure Features
    df['dep_hour'] = df['departure_time'].dt.hour
    df['dep_day'] = df['departure_time'].dt.day
    df['dep_month'] = df['departure_time'].dt.month
    df['dep_dayofweek'] = df['departure_time'].dt.dayofweek + 1  # Make it 1 (Mon) to 7 (Sun)
    df['dep_quarter'] = df['departure_time'].dt.quarter
    df['dep_season'] = df['dep_month'].apply(get_season)
    df['dep_is_weekend'] = df['dep_dayofweek'].isin([6, 7]).astype(int)
    df['dep_time_of_day'] = df['dep_hour'].apply(get_time_of_day)

    # Arrival Features
    df['arr_hour'] = df['arrival_time'].dt.hour
    df['arr_day'] = df['arrival_time'].dt.day
    df['arr_month'] = df['arrival_time'].dt.month
    df['arr_dayofweek'] = df['arrival_time'].dt.dayofweek + 1
    df['arr_quarter'] = df['arrival_time'].dt.quarter
    df['arr_season'] = df['arr_month'].apply(get_season)
    df['arr_is_weekend'] = df['arr_dayofweek'].isin([6, 7]).astype(int)
    df['arr_time_of_day'] = df['arr_hour'].apply(get_time_of_day)

    return df

In [ ]:
def add_route_column(df):
    df['route'] = df['departure_point'].astype(str) + ' → ' + df['arrival_point'].astype(str)
    return df

In [ ]:
def add_weather_features(df):
    df['departure_time'] = pd.to_datetime(df['departure_time'], errors='coerce')
    df['arrival_time'] = pd.to_datetime(df['arrival_time'], errors='coerce')

    df['departure_date'] = df['departure_time'].dt.date
    df['arrival_date'] = df['arrival_time'].dt.date

    # STEP 2: Load IATA → coordinates from OurAirports
    airports_df = pd.read_csv("data/airports.csv")  # From https://ourairports.com/data/
    iata_df = airports_df[['iata_code', 'latitude_deg', 'longitude_deg']].dropna()
    iata_coords = dict(zip(iata_df['iata_code'], zip(iata_df['latitude_deg'], iata_df['longitude_deg'])))

    # STEP 3: Find closest station for each IATA code (only once)
    unique_iatas = set(df['departure_point'].unique()) | set(df['arrival_point'].unique())
    iata_station = {}

    print("🔍 Finding nearest stations for all unique airports...")
    for iata in tqdm(unique_iatas, desc="Mapping IATA to Station"):
        coord = iata_coords.get(iata)
        if not coord:
            iata_station[iata] = None
            continue
        lat, lon = coord
        try:
            stations = Stations().nearby(lat, lon)
            station = stations.fetch(1)
            if not station.empty:
                iata_station[iata] = station.index[0]
            else:
                iata_station[iata] = None
        except:
            iata_station[iata] = None
        time.sleep(1)
    # STEP 4: Create weather fetch helper
    def fetch_weather(iata, date):
        station_id = iata_station.get(iata)
        if not station_id or pd.isna(date):
            return None
        date = pd.to_datetime(date)
        try:
            weather = Daily(station_id, date, date).fetch()
            if not weather.empty:
                w = weather.iloc[0]
                return {
                    'temp': w['tavg'],
                    'precip': w['prcp'],
                    'wind': w['wspd'],
                    'snow': w['snow'],
                    'weather_code': w.get('coco')
                }
        except:
            return None
        return None

    # STEP 5: Build unique (iata, date) pairs for both dep/arr
    dep_requests = df[['departure_point', 'departure_date']].dropna().drop_duplicates()
    arr_requests = df[['arrival_point', 'arrival_date']].dropna().drop_duplicates()

    # Convert to records and tag
    dep_requests['type'] = 'dep'
    arr_requests['type'] = 'arr'
    dep_requests.columns = ['iata', 'date', 'type']
    arr_requests.columns = ['iata', 'date', 'type']
    all_requests = pd.concat([dep_requests, arr_requests]).drop_duplicates()

    # STEP 6: Fetch weather for all combinations
    weather_records = []
    print("🌦 Fetching weather data for all airport-date combinations...")
    for _, row in tqdm(all_requests.iterrows(), total=all_requests.shape[0], desc="Weather Fetching"):
        iata = row['iata']
        date = row['date']
        w = fetch_weather(iata, date)
        if w:
            weather_records.append({
                'iata': iata,
                'date': date,
                'type': row['type'],
                **w
            })

    weather_df = pd.DataFrame(weather_records)
    # STEP 7: Merge weather back into main DataFrame
    # Separate dep and arr weather
    dep_weather = weather_df[weather_df['type'] == 'dep'].drop(columns='type')
    arr_weather = weather_df[weather_df['type'] == 'arr'].drop(columns='type')

    dep_weather.columns = ['departure_point', 'departure_date'] + [f'dep_{col}' for col in dep_weather.columns[2:]]
    arr_weather.columns = ['arrival_point', 'arrival_date'] + [f'arr_{col}' for col in arr_weather.columns[2:]]

    df = df.merge(dep_weather, on=['departure_point', 'departure_date'], how='left')
    df = df.merge(arr_weather, on=['arrival_point', 'arrival_date'], how='left')

    # STEP 8: Save output
    return df

In [ ]:
def immute_weather_features(df):
    # 1. Remove weather code columns
    df = df.drop(columns=['dep_weather_code', 'arr_weather_code','dep_snow','arr_snow'], errors='ignore')

    # 2. Fill snow columns with 0 (assume no snow if not reported)
    # df['dep_snow'] = df['dep_snow'].fillna(0)
    # df['arr_snow'] = df['arr_snow'].fillna(0)

    # 3. TEMP — Impute by month and location
    df['dep_temp'] = df.groupby(['departure_point', 'dep_month'])['dep_temp'].transform(lambda x: x.fillna(x.median()))
    df['arr_temp'] = df.groupby(['arrival_point', 'arr_month'])['arr_temp'].transform(lambda x: x.fillna(x.median()))

    # 4. WIND — Impute by hour and location
    df['dep_wind'] = df.groupby(['departure_point', 'dep_hour'])['dep_wind'].transform(lambda x: x.fillna(x.median()))
    df['arr_wind'] = df.groupby(['arrival_point', 'arr_hour'])['arr_wind'].transform(lambda x: x.fillna(x.median()))

    # 5. PRECIP — Impute by time_of_day and location
    df['dep_precip'] = df.groupby(['departure_point', 'dep_time_of_day'])['dep_precip'].transform(lambda x: x.fillna(x.median()))
    df['arr_precip'] = df.groupby(['arrival_point', 'arr_time_of_day'])['arr_precip'].transform(lambda x: x.fillna(x.median()))

    # 6. Fallbacks
    df['dep_precip'] = df['dep_precip'].fillna(0)
    df['arr_precip'] = df['arr_precip'].fillna(0)

    for col in ['dep_temp', 'dep_wind', 'arr_temp', 'arr_wind']:
        df[col] = df[col].fillna(df[col].median())
    return df

In [ ]:
def prepared_train_data(final_df):
    final_df = add_column_duration(fd_train)
    ## Select records which has less than 20hrs long duration
    ## Filter records which have delay(target) between 1minute to 3hrs
    final_df = add_datetime_features(final_df)
    final_df = add_route_column(final_df)
    final_df = fd_train[fd_train["duration"] < 68400]
    df_non_zeros_without_outliers = final_df[(final_df['target'] > 0) & (final_df['target'] < 165)]
    all_zeros = final_df[final_df['target'] == 0]

    # Create a copy of all_zeros and assign random negative values between -1 and -165 to 'target'
    all_zeros_negative = all_zeros.copy()
    positive_targets = df_non_zeros_without_outliers[df_non_zeros_without_outliers['target'] > 0]['target'].values

    all_zeros_negative['target'] = -np.random.choice(positive_targets, size=len(all_zeros_negative), replace=True)
    all_zeros_pos_neg = pd.concat([all_zeros_negative, df_non_zeros_without_outliers], axis=0, ignore_index=True)
    all_zeros_pos_neg.drop(["id", "flight_id", "date"], inplace=True, axis=1)
    all_zeros_pos_neg = add_weather_features(all_zeros_pos_neg)
    all_zeros_pos_neg = immute_weather_features(all_zeros_pos_neg)
    return all_zeros_pos_neg


In [ ]:
df = prepared_train_data(fd_train)
df

In [ ]:
df.describe()

In [ ]:
df.to_csv("data/preprocessed_train_data.csv", index=False)

In [ ]:
def prepared_test_data(final_test_df):
    final_test_df.drop(["flight_id", "date"], inplace=True, axis=1)
    final_df = add_datetime_features(final_test_df)
    final_df = add_route_column(final_df)
    final_df = add_column_duration(final_df)
    final_df = add_weather_features(final_df)
    final_df = immute_weather_features(final_df)
    return final_df

In [ ]:
df_test = prepared_test_data(fd_test)

df_test.to_csv("data/preprocessed_test_data.csv", index=False)